In [ ]:
import pandas as pd
import io
import os

In [ ]:
#Environment variables
WEATHER_DATA_DIR = 'Database/Weather Data'


In [ ]:
def split_csv_by_empty_row(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Find the empty row
    empty_row_indices = [i for i, line in enumerate(lines) if line.strip() == '']

    # Assuming there's only one empty row separating the tables
    separator_index = empty_row_indices[0]

    # Split the lines into two lists based on the separator index
    table1_lines = lines[:separator_index]
    table2_lines = lines[separator_index + 1:]

    # Convert the lists of lines to DataFrames
    metadata_df = pd.read_csv(io.StringIO(''.join(table1_lines)))
    readings_df = pd.read_csv(io.StringIO(''.join(table2_lines)))

    return metadata_df, readings_df


In [ ]:
# Split the CSV into two DataFrames
metadata_df, readings_df = split_csv_by_empty_row(file_path)

In [ ]:
# Empty list that will be populated with the datasets
all_dfs = []

# Loops through the files in the Datasets directory
for file in os.listdir(WEATHER_DATA_DIR):

    # Executes only the .csv files
    if(file.endswith('.csv')):
        # Split the CSV into two DataFrames
        metadata_df, readings_df = split_csv_by_empty_row(f'{WEATHER_DATA_DIR}/{file}')
        
        # Create new columns in readings_df
        readings_df['latitude'] = metadata_df['latitude']
        readings_df['longitude'] = metadata_df['longitude']
        readings_df['elevation'] = metadata_df['elevation']
        readings_df['timezone'] = metadata_df['timezone']
        readings_df['timezone_abbreviation'] = metadata_df['timezone_abbreviation']

        # Merge metadata_df and readings_df based on location_id
        merged_df = pd.merge(readings_df, metadata_df, on='location_id')

        # Drop the reduncant columns
        merged_df.drop(columns=['latitude_y', 'longitude_y', 'elevation_y', 'timezone_y', 'timezone_abbreviation_y'], inplace=True)

        all_dfs.append(merged_df)

all_dfs[0]
